In [1]:
import requests
import re
from bs4 import BeautifulSoup, Comment
import pandas as pd
import numpy as np
from io import StringIO  
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from datetime import datetime

In [8]:
# url = 'https://www.baseball-reference.com/leagues/majors/2018-standard-batting.shtml'
url = 'https://www.baseball-reference.com/leagues/majors'
# url_league = 'https://www.baseball-reference.com/players/l/lindofr01.shtml'
r = requests.get(url)
r.status_code

200

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')
soup.title.string

In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

urls = [
    'https://www.baseball-reference.com/leagues/majors/2018-standard-batting.shtml',
    'https://www.baseball-reference.com/leagues/majors/2019-standard-batting.shtml',
    'https://www.baseball-reference.com/leagues/majors/2020-standard-batting.shtml',
    'https://www.baseball-reference.com/leagues/majors/2021-standard-batting.shtml',
    'https://www.baseball-reference.com/leagues/majors/2022-standard-batting.shtml',
    'https://www.baseball-reference.com/leagues/majors/2023-standard-batting.shtml',
    'https://www.baseball-reference.com/leagues/majors/2024-standard-batting.shtml',
    'https://www.baseball-reference.com/leagues/majors/2025-standard-batting.shtml',
]

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}

# EXACT column order you want (no spaces, matches real table)
columns = [
    'Rk','Player','Age','Team','Lg','WAR','G','PA','AB','R','H','2B','3B','HR','RBI',
    'SB','CS','BB','SO','BA','OBP','SLG','OPS','OPS+','rOBA','Rbat+','TB','GIDP',
    'HBP','SH','SF','IBB','Pos','Awards'  # <-- Awards has space at end on some years
]

all_data = []

for url in urls:
    year = int(re.search(r'/(\d{4})', url).group(1))
    print(f"\nScraping {year}...")

    r = requests.get(url, headers=headers, timeout=15)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, 'html.parser')

    div = soup.find('div', id='switcher_players_standard_batting')
    if not div:
        print("  Switcher div not found")
        continue
    table = div.find('table', id='players_standard_batting')
    if not table:
        print("  Table not found")
        continue
    tbody = table.find('tbody')
    if not tbody:
        print("  Tbody not found")
        continue

    seen = set()

    for row in tbody.find_all('tr'):
        if row.get('class') and 'thead' in row.get('class'):
            continue

        cells = row.find_all(['th', 'td'])
        if len(cells) < 25:
            continue

        # Player name (cell 1)
        player_cell = cells[1]
        # player_name = player_cell.get_text(strip=True).rstrip('*#')
        player_name = player_cell.get_text(strip=True)
        team = cells[3].get_text(strip=True)

        # Also keep the link
        a = player_cell.find('a')
        if a:
            row_dict['Player_Link'] = 'https://www.baseball-reference.com' + a['href']

        # Only keep TOT row (or first row if no TOT)
        if player_name in seen and team != 'TOT':
            continue
        if team == 'TOT' or player_name not in seen:
            seen.add(player_name)

            row_dict = {'Year': year}

            # Map by fixed position (this works 100% across all years)
            pos_map = {
                0: 'Rk', 1: 'Player', 2: 'Age', 3: 'Team', 4: 'Lg',
                5: 'WAR', 6: 'G', 7: 'PA', 8: 'AB', 9: 'R', 10: 'H',
                11: '2B', 12: '3B', 13: 'HR', 14: 'RBI', 15: 'SB',
                16: 'CS', 17: 'BB', 18: 'SO', 19: 'BA', 20: 'OBP',
                21: 'SLG', 22: 'OPS', 23: 'OPS+', 24: 'rOBA', 25: 'Rbat+',
                26: 'TB', 27: 'GIDP', 28: 'HBP', 29: 'SH', 30: 'SF',
                31: 'IBB', 32: 'Pos', 33: 'Awards'  # Awards is sometimes last
            }

            for idx, cell in enumerate(cells):
                col_name = pos_map.get(idx)
                if not col_name:
                    continue
                if col_name == 'Player':
                    row_dict['Player'] = player_name
                    a = cell.find('a')
                    if a:
                        row_dict['Player_Link'] = 'https://www.baseball-reference.com' + a['href']
                elif col_name == 'Awards':
                    row_dict['Awards'] = cell.get_text(strip=True)
                else:
                    row_dict[col_name] = cell.get_text(strip=True)

            # Ensure all columns exist
            for col in columns:
                row_dict.setdefault(col, '')

            all_data.append(row_dict)

    print(f"  {len(seen)} players scraped for {year}")

    time.sleep(1.2)

# Final DataFrame
df = pd.DataFrame(all_data, columns=['Year'] + columns + ['Player_Link'])

# Convert numeric
numeric_cols = ['Age','WAR','G','PA','AB','R','H','2B','3B','HR','RBI','SB','CS','BB','SO','OPS+','TB','GIDP','HBP','SH','SF','IBB','Rbat+']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df = df.sort_values(['Year', 'Rk'], ascending=[False, True]).reset_index(drop=True)

print("\nDONE! Perfect data.")
print(f"Total players: {len(df)}")
print("2025 players:", len(df[df.Year == 2025]))
print("2024 players:", len(df[df.Year == 2024]))

print("\n2025 Top 10:")
print(df[df.Year == 2025][['Rk','Player','Team','WAR','G','PA','HR','OPS','OPS+','rOBA','Rbat+']].head(10))

df.to_csv('MLB_2018_2025_Perfect.csv', index=False)
print("\nSaved: MLB_2018_2025_Perfect.csv")


Scraping 2018...
  1272 players scraped for 2018

Scraping 2019...
  1286 players scraped for 2019

Scraping 2020...
  619 players scraped for 2020

Scraping 2021...
  1373 players scraped for 2021

Scraping 2022...
  790 players scraped for 2022

Scraping 2023...
  764 players scraped for 2023

Scraping 2024...
  742 players scraped for 2024

Scraping 2025...
  764 players scraped for 2025

DONE! Perfect data.
Total players: 7610
2025 players: 764
2024 players: 742

2025 Top 10:
    Rk             Player Team  WAR      G     PA    HR   OPS   OPS+  rOBA  \
0             MLB Average       NaN    NaN    NaN   NaN  .719  100.0  .319   
1    1  Francisco Lindor#  NYM  5.9  160.0  732.0  31.0  .811  129.0  .359   
2   10    Randy Arozarena  SEA  4.0  160.0  709.0  27.0  .760  119.0  .341   
3  100      Bryson Stott*  PHI  2.2  147.0  560.0  13.0  .719   96.0  .324   
4  101  Mike Yastrzemski*  2TM  2.8  146.0  558.0  17.0  .735  109.0  .328   
5  102       Ketel Marte#  ARI  4.4  126.0  55

In [ ]:
# read in MLB_2018_2025_Perfect.csv 
# get *# or nothing or ? and make batting hand column
# strip those characters from player names
# get rid of rows that say mlb average in play name (# # 1. Remove league average rows)
# # 2. Keep only players with 100+ Plate Appearances

In [54]:
mlb = pd.read_csv('MLB_2018_2025_Perfect.csv')
mlb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7610 entries, 0 to 7609
Data columns (total 36 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Year         7610 non-null   int64  
 1   Rk           7602 non-null   float64
 2   Player       7610 non-null   object 
 3   Age          7602 non-null   float64
 4   Team         7602 non-null   object 
 5   Lg           7602 non-null   object 
 6   WAR          7600 non-null   float64
 7   G            7602 non-null   float64
 8   PA           7602 non-null   float64
 9   AB           7602 non-null   float64
 10  R            7602 non-null   float64
 11  H            7602 non-null   float64
 12  2B           7602 non-null   float64
 13  3B           7602 non-null   float64
 14  HR           7602 non-null   float64
 15  RBI          7602 non-null   float64
 16  SB           7602 non-null   float64
 17  CS           7602 non-null   float64
 18  BB           7602 non-null   float64
 19  SO    

In [55]:
# get rid of rows that say mlb average in play name (# # 1. Remove league average rows)
mlb = mlb[~mlb['Player'].str.contains('MLB Average', case=False, na=False)]
mlb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7602 entries, 1 to 7609
Data columns (total 36 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Year         7602 non-null   int64  
 1   Rk           7602 non-null   float64
 2   Player       7602 non-null   object 
 3   Age          7602 non-null   float64
 4   Team         7602 non-null   object 
 5   Lg           7602 non-null   object 
 6   WAR          7600 non-null   float64
 7   G            7602 non-null   float64
 8   PA           7602 non-null   float64
 9   AB           7602 non-null   float64
 10  R            7602 non-null   float64
 11  H            7602 non-null   float64
 12  2B           7602 non-null   float64
 13  3B           7602 non-null   float64
 14  HR           7602 non-null   float64
 15  RBI          7602 non-null   float64
 16  SB           7602 non-null   float64
 17  CS           7602 non-null   float64
 18  BB           7602 non-null   float64
 19  SO         

In [56]:
# # 2. Keep only players with 100+ Plate Appearances
mlb = mlb[mlb['PA'] >= 100]
mlb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3518 entries, 1 to 7599
Data columns (total 36 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Year         3518 non-null   int64  
 1   Rk           3518 non-null   float64
 2   Player       3518 non-null   object 
 3   Age          3518 non-null   float64
 4   Team         3518 non-null   object 
 5   Lg           3518 non-null   object 
 6   WAR          3518 non-null   float64
 7   G            3518 non-null   float64
 8   PA           3518 non-null   float64
 9   AB           3518 non-null   float64
 10  R            3518 non-null   float64
 11  H            3518 non-null   float64
 12  2B           3518 non-null   float64
 13  3B           3518 non-null   float64
 14  HR           3518 non-null   float64
 15  RBI          3518 non-null   float64
 16  SB           3518 non-null   float64
 17  CS           3518 non-null   float64
 18  BB           3518 non-null   float64
 19  SO         

In [57]:
# make column 'batting_hand' that learns from symbols attached to the end of player names, such as:
# get special character at end of player name which is one of these options:
# '*' = both
# '#' = left
# ''= right
# '?' = unknown
#save column 'batting_hand'
# strip those characters from player names
def get_batting_hand(name):
    if name.endswith('*'):
        return 'both'
    elif name.endswith('#'):
        return 'left'
    elif name.endswith('?'):
        return 'unknown'
    else:
        return 'right'
mlb['batting_hand'] = mlb['Player'].apply(get_batting_hand)
mlb['Player'] = mlb['Player'].str.rstrip('*#?')
mlb.head()

,Year,Rk,Player,Age,Team,Lg,WAR,G,PA,AB,...,TB,GIDP,HBP,SH,SF,IBB,Pos,Awards,Player_Link,batting_hand
1,2025,1.0,Francisco Lindor,31.0,NYM,NL,5.9,160.0,732.0,644.0,...,300.0,9.0,16.0,0.0,7.0,2.0,*6/DH,"AS,MVP-10",https://www.baseball-reference.com/players/d/d...,left
2,2025,10.0,Randy Arozarena,30.0,SEA,AL,4.0,160.0,709.0,613.0,...,261.0,9.0,27.0,0.0,5.0,2.0,*7/D,AS,https://www.baseball-reference.com/players/r/r...,right
3,2025,100.0,Bryson Stott,27.0,PHI,NL,2.2,147.0,560.0,499.0,...,195.0,4.0,1.0,2.0,4.0,1.0,*46H,NaN,https://www.baseball-reference.com/players/y/y...,both
4,2025,101.0,Mike Yastrzemski,34.0,2TM,2LG,2.8,146.0,558.0,477.0,...,192.0,7.0,2.0,2.0,5.0,4.0,*9H8/7D1,NaN,https://www.baseball-reference.com/players/m/m...,both
5,2025,102.0,Ketel Marte,31.0,ARI,NL,4.4,126.0,556.0,480.0,...,248.0,12.0,9.0,0.0,3.0,6.0,4D/H,"AS,MVP-22,SS",https://www.baseball-reference.com/players/t/t...,left


In [58]:
#remove awards column
mlb = mlb.drop(columns=['Awards'])
# for each year, order by rank ascending
mlb = mlb.sort_values(['Year', 'Rk'], ascending=[True, True]).reset_index(drop=True)
mlb.head()

,Year,Rk,Player,Age,Team,Lg,WAR,G,PA,AB,...,Rbat+,TB,GIDP,HBP,SH,SF,IBB,Pos,Player_Link,batting_hand
0,2018,1.0,Francisco Lindor,24.0,CLE,AL,7.3,158.0,745.0,661.0,...,133.0,343.0,5.0,8.0,3.0,3.0,7.0,*6/D,https://www.baseball-reference.com/players/t/t...,left
1,2018,2.0,Trea Turner,25.0,WSN,NL,4.9,162.0,740.0,664.0,...,105.0,276.0,7.0,5.0,2.0,0.0,3.0,*6/H,https://www.baseball-reference.com/players/m/m...,right
2,2018,3.0,Manny Machado,25.0,2TM,2LG,6.2,162.0,709.0,632.0,...,142.0,340.0,26.0,2.0,0.0,5.0,18.0,*65,https://www.baseball-reference.com/players/h/h...,right
3,2018,4.0,CÃ©sar HernÃ¡ndez,28.0,PHI,NL,2.1,161.0,708.0,605.0,...,100.0,219.0,12.0,4.0,1.0,3.0,4.0,*4/HD,https://www.baseball-reference.com/players/f/f...,left
4,2018,5.0,Freddie Freeman,28.0,ATL,NL,5.5,162.0,707.0,618.0,...,144.0,312.0,11.0,7.0,0.0,6.0,12.0,*3/H,https://www.baseball-reference.com/players/m/m...,both


In [68]:
# order by player and year
mlb = mlb.sort_values(['Player', 'Year'], ascending=[True, True])
mlb

,Year,Rk,Player,Age,Team,Lg,WAR,G,PA,AB,...,Rbat+,TB,GIDP,HBP,SH,SF,IBB,Pos,Player_Link,batting_hand
366,2018,367.0,A.J. Ellis,37.0,SDP,NL,0.2,66.0,183.0,151.0,...,105.0,52.0,2.0,1.0,3.0,2.0,1.0,2H/7D,https://www.baseball-reference.com/players/n/n...,right
173,2018,174.0,AJ Pollock,30.0,ARI,NL,2.2,113.0,460.0,413.0,...,103.0,200.0,6.0,8.0,1.0,7.0,2.0,*8/H,https://www.baseball-reference.com/players/m/m...,right
696,2019,249.0,AJ Pollock,31.0,LAD,NL,0.0,86.0,342.0,308.0,...,104.0,144.0,7.0,7.0,0.0,4.0,1.0,87/HD,https://www.baseball-reference.com/players/s/s...,right
1001,2020,103.0,AJ Pollock,32.0,LAD,NL,0.8,55.0,210.0,196.0,...,127.0,111.0,6.0,0.0,0.0,2.0,1.0,*78D/H,https://www.baseball-reference.com/players/r/r...,right
1382,2021,174.0,AJ Pollock,33.0,LAD,NL,3.0,117.0,422.0,384.0,...,133.0,206.0,4.0,6.0,0.0,2.0,4.0,7H/8,https://www.baseball-reference.com/players/s/s...,right
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2990,2024,389.0,Ãngel MartÃ­nez,22.0,CLE,AL,0.1,43.0,169.0,151.0,...,80.0,51.0,2.0,0.0,1.0,2.0,0.0,87/9H5D4,https://www.baseball-reference.com/players/t/t...,left
3211,2025,155.0,Ãngel MartÃ­nez,23.0,CLE,AL,-0.7,139.0,484.0,446.0,...,68.0,160.0,8.0,6.0,5.0,4.0,0.0,*84H/97,https://www.baseball-reference.com/players/m/m...,left
597,2019,150.0,Ãscar Mercado,24.0,CLE,AL,2.2,115.0,482.0,438.0,...,95.0,194.0,9.0,5.0,7.0,4.0,0.0,*87/9HD,https://www.baseball-reference.com/players/n/n...,right
1533,2021,325.0,Ãscar Mercado,26.0,CLE,AL,1.0,72.0,238.0,214.0,...,94.0,79.0,1.0,2.0,1.0,0.0,0.0,79H/8,https://www.baseball-reference.com/players/v/v...,right


In [64]:
# how many unique players that have data for all years 2018-2025?
mlb['Player'].nunique()
player_counts = mlb['Player'].value_counts()
players_all_years = player_counts[player_counts == 8].index
print(f"Players with data for all years: {len(players_all_years)}")

Players with data for all years: 83


In [71]:
# how many players have more than 1 row for the same year?
# duplicates = for year in years check to see if any player has more than 1 row
duplicates = mlb.duplicated(subset=['Player', 'Year'], keep=False)
duplicate_rows = mlb[duplicates]
print(f"Players with duplicate year entries: {duplicate_rows['Player'].nunique()}")
# what players are they?
duplicate_rows[['Player', 'Year', 'Team']].sort_values(['Player', 'Year'])

Players with duplicate year entries: 1


,Player,Year,Team
3279,Max Muncy,2025,LAD
3381,Max Muncy,2025,ATH


In [65]:
# save mlb as cleaned csv
mlb.to_csv('MLB_2018_2025_Cleaned.csv', index=False)